# 🏔️ Function 1: Calculate Topographic Metrics

## Building the `calculate_topographic_metrics` Function

**Learning Objectives:**
- Master terrain analysis using digital elevation models (DEMs)
- Understand topographic derivatives and their applications
- Learn to calculate slope, aspect, hillshade, and curvature
- Implement robust terrain analysis workflows
- Handle edge cases and data quality issues in elevation data
- Create publication-quality terrain visualizations

**Professional Context:**
Topographic analysis is fundamental to many geospatial applications. Professionals use these techniques for:
- Hydrological modeling and watershed analysis
- Landslide and erosion risk assessment
- Solar radiation and viewshed analysis
- Habitat modeling and species distribution
- Infrastructure planning and site selection

## 🎯 Function Overview

**Function Signature:**
```python
def calculate_topographic_metrics(dem_path, metrics=['slope', 'aspect', 'hillshade'], 
                                 output_dir=None, cell_size=None, 
                                 hillshade_azimuth=315, hillshade_altitude=45):
    """
    Calculate comprehensive topographic metrics from a digital elevation model.
    
    Parameters:
    -----------
    dem_path : str
        Path to the input DEM raster file
    metrics : list, optional
        List of metrics to calculate ['slope', 'aspect', 'hillshade', 'curvature']
        Default: ['slope', 'aspect', 'hillshade']
    output_dir : str, optional
        Directory to save output rasters. If None, returns arrays only
    cell_size : float, optional
        Cell size for calculations. If None, uses DEM resolution
    hillshade_azimuth : float, optional
        Sun azimuth angle for hillshade (0-360 degrees)
        Default: 315 (northwest)
    hillshade_altitude : float, optional
        Sun altitude angle for hillshade (0-90 degrees)
        Default: 45 degrees
    
    Returns:
    --------
    dict
        Dictionary containing calculated metrics, arrays, and metadata
    """
```

**Key Capabilities:**
- 📐 Calculate slope (gradient) in degrees and percent
- 🧭 Calculate aspect (direction) in degrees from north
- 🌄 Generate realistic hillshade with customizable sun position
- 📈 Calculate curvature (profile and planform)
- 🗺️ Handle coordinate system transformations for accurate calculations
- 💾 Support both in-memory processing and file output

## 🏗️ Implementation Strategy

Our topographic metrics function will follow this workflow:

### Step 1: DEM Loading and Validation
```python
# Load DEM and validate data quality
with rasterio.open(dem_path) as src:
    elevation = src.read(1)
    profile = src.profile
    transform = src.transform
    
# Handle nodata and fill gaps if necessary
elevation_filled = fill_nodata_gaps(elevation, nodata_value)
```

### Step 2: Calculate Gradients
```python
# Calculate gradients using numpy gradient
dy, dx = np.gradient(elevation, cell_size, cell_size)

# Calculate slope magnitude
slope_radians = np.arctan(np.sqrt(dx**2 + dy**2))
slope_degrees = np.degrees(slope_radians)
```

### Step 3: Terrain Derivatives
```python
# Calculate aspect (direction of steepest slope)
aspect_radians = np.arctan2(-dx, dy)
aspect_degrees = np.degrees(aspect_radians) % 360

# Calculate hillshade using sun position
hillshade = calculate_hillshade(slope_radians, aspect_radians, 
                               sun_azimuth, sun_altitude)
```

## 🚀 Hands-On Example: Building the Function

Let's build the complete topographic metrics function step by step:

In [ ]:
import rasterio
import numpy as np
import os
from scipy import ndimage
from scipy.interpolate import griddata
import matplotlib.pyplot as plt
from matplotlib.colors import LightSource
import warnings
warnings.filterwarnings('ignore')

def calculate_topographic_metrics(dem_path, metrics=['slope', 'aspect', 'hillshade'], 
                                 output_dir=None, cell_size=None, 
                                 hillshade_azimuth=315, hillshade_altitude=45):
    """
    Calculate comprehensive topographic metrics from a digital elevation model.
    """
    
    print(f"🏔️ Starting topographic metrics calculation...")
    print(f"   📁 Input DEM: {os.path.basename(dem_path)}")
    print(f"   📊 Metrics to calculate: {metrics}")
    
    # Step 1: Input validation
    if not os.path.exists(dem_path):
        raise FileNotFoundError(f"DEM file not found: {dem_path}")
    
    valid_metrics = ['slope', 'aspect', 'hillshade', 'curvature']
    invalid_metrics = [m for m in metrics if m not in valid_metrics]
    if invalid_metrics:
        raise ValueError(f"Invalid metrics: {invalid_metrics}. Valid options: {valid_metrics}")
    
    # Step 2: Load DEM data
    with rasterio.open(dem_path) as src:
        elevation = src.read(1, masked=True)  # Read as masked array
        profile = src.profile.copy()
        transform = src.transform
        crs = src.crs
        bounds = src.bounds
        nodata = src.nodata
        
        print(f"   📊 DEM info:")
        print(f"     Shape: {elevation.shape}")
        print(f"     CRS: {crs}")
        print(f"     Resolution: {transform.a:.2f} x {abs(transform.e):.2f}")
        print(f"     Elevation range: {elevation.min():.1f} - {elevation.max():.1f}")
        
        # Determine cell size from transform if not provided
        if cell_size is None:
            cell_size = abs(transform.a)  # Assume square pixels
            print(f"     Using DEM resolution: {cell_size:.2f} units")
        else:
            print(f"     Using specified cell size: {cell_size:.2f} units")
    
    # Step 3: Handle nodata values
    if np.ma.is_masked(elevation):
        valid_pixels = np.ma.count(elevation)
        total_pixels = elevation.size
        print(f"   📈 Valid pixels: {valid_pixels:,} ({100*valid_pixels/total_pixels:.1f}%)")
        
        # Fill small gaps for better gradient calculation
        if elevation.mask.sum() / elevation.size < 0.1:  # Less than 10% missing
            print(f"   🔧 Filling small data gaps for gradient calculation...")
            elevation_filled = elevation.filled(np.nan)
            
            # Simple interpolation for small gaps
            mask = ~np.isnan(elevation_filled)
            if mask.sum() > 10:  # Need at least some points
                coords = np.array(np.where(mask)).T
                values = elevation_filled[mask]
                
                # Interpolate to fill gaps
                y_indices, x_indices = np.mgrid[0:elevation.shape[0], 0:elevation.shape[1]]
                missing_coords = np.array(np.where(~mask)).T
                
                if len(missing_coords) > 0 and len(coords) > 3:
                    try:
                        interpolated = griddata(coords, values, missing_coords, method='linear', fill_value=np.nan)
                        elevation_filled[~mask] = interpolated
                        elevation = np.ma.masked_invalid(elevation_filled)
                        print(f"     ✅ Filled {len(missing_coords)} pixels")
                    except:
                        print(f"     ⚠️ Could not interpolate gaps, using original data")
        else:
            elevation = elevation.filled(np.nan)
    else:
        elevation = np.ma.masked_invalid(elevation)
    
    # Step 4: Calculate gradients
    print(f"   📐 Calculating gradients...")
    
    # Use numpy gradient with proper cell size
    # Note: gradient returns [dy, dx] for 2D arrays
    elevation_array = elevation.filled(np.nan) if np.ma.is_masked(elevation) else elevation
    
    # Calculate gradients (rise over run)
    dy, dx = np.gradient(elevation_array, cell_size, cell_size)
    
    # Mask gradients where elevation is masked
    if np.ma.is_masked(elevation):
        dy = np.ma.masked_where(elevation.mask, dy)
        dx = np.ma.masked_where(elevation.mask, dx)
    
    # Initialize results dictionary
    results = {
        'metrics_calculated': [],
        'arrays': {},
        'statistics': {},
        'metadata': {
            'dem_path': dem_path,
            'shape': elevation.shape,
            'cell_size': cell_size,
            'crs': str(crs),
            'bounds': bounds,
            'elevation_range': (float(elevation.min()), float(elevation.max())),
            'nodata': nodata
        }
    }
    
    # Step 5: Calculate individual metrics
    if 'slope' in metrics:
        print(f"   🔺 Calculating slope...")
        
        # Slope in radians
        slope_radians = np.arctan(np.sqrt(dx**2 + dy**2))
        
        # Convert to degrees
        slope_degrees = np.degrees(slope_radians)
        
        # Convert to percent
        slope_percent = np.tan(slope_radians) * 100
        
        # Store results
        results['arrays']['slope_degrees'] = slope_degrees
        results['arrays']['slope_percent'] = slope_percent
        results['arrays']['slope_radians'] = slope_radians  # Keep for other calculations
        
        # Calculate statistics
        if np.ma.is_masked(slope_degrees):
            valid_slope = slope_degrees.compressed()
        else:
            valid_slope = slope_degrees[~np.isnan(slope_degrees)]
        
        if len(valid_slope) > 0:
            results['statistics']['slope'] = {
                'mean_degrees': float(np.mean(valid_slope)),
                'max_degrees': float(np.max(valid_slope)),
                'std_degrees': float(np.std(valid_slope)),
                'median_degrees': float(np.median(valid_slope))
            }
        
        results['metrics_calculated'].append('slope')
        print(f"     ✅ Slope range: {valid_slope.min():.1f}° - {valid_slope.max():.1f}°")
    
    if 'aspect' in metrics:
        print(f"   🧭 Calculating aspect...")
        
        # Calculate aspect in radians (direction of steepest slope)
        # Note: atan2(-dx, dy) gives direction of steepest uphill slope
        # We want downhill slope direction, so use atan2(dx, -dy)
        aspect_radians = np.arctan2(-dx, dy)
        
        # Convert to degrees (0-360, with 0 = North, 90 = East)
        aspect_degrees = (np.degrees(aspect_radians) + 360) % 360
        
        # Handle flat areas (set to -1 or NaN)
        if 'slope' in results['arrays']:
            flat_areas = results['arrays']['slope_degrees'] < 1.0  # Less than 1 degree
            aspect_degrees = np.where(flat_areas, -1, aspect_degrees)
        
        results['arrays']['aspect_degrees'] = aspect_degrees
        results['arrays']['aspect_radians'] = aspect_radians  # Keep for hillshade
        
        # Calculate statistics (excluding flat areas)
        if np.ma.is_masked(aspect_degrees):
            valid_aspect = aspect_degrees.compressed()
        else:
            valid_aspect = aspect_degrees[~np.isnan(aspect_degrees)]
        
        valid_aspect = valid_aspect[valid_aspect >= 0]  # Exclude flat areas (-1)
        
        if len(valid_aspect) > 0:
            # Circular statistics for aspect
            aspect_radians_valid = np.radians(valid_aspect)
            mean_x = np.mean(np.cos(aspect_radians_valid))
            mean_y = np.mean(np.sin(aspect_radians_valid))
            mean_direction = np.degrees(np.arctan2(mean_y, mean_x)) % 360
            
            results['statistics']['aspect'] = {
                'mean_direction_degrees': float(mean_direction),
                'concentration': float(np.sqrt(mean_x**2 + mean_y**2)),
                'flat_areas_percent': float(100 * np.sum(aspect_degrees == -1) / aspect_degrees.size)
            }
        
        results['metrics_calculated'].append('aspect')
        print(f"     ✅ Mean aspect direction: {mean_direction:.1f}° from North")
    
    if 'hillshade' in metrics:
        print(f"   🌄 Calculating hillshade...")
        
        # Need slope and aspect for hillshade
        if 'slope_radians' not in results['arrays']:
            slope_radians = np.arctan(np.sqrt(dx**2 + dy**2))
        else:
            slope_radians = results['arrays']['slope_radians']
        
        if 'aspect_radians' not in results['arrays']:
            aspect_radians = np.arctan2(-dx, dy)
        else:
            aspect_radians = results['arrays']['aspect_radians']
        
        # Convert sun position to radians
        sun_azimuth_rad = np.radians(hillshade_azimuth)
        sun_altitude_rad = np.radians(hillshade_altitude)
        
        # Calculate hillshade using the standard formula
        hillshade = (
            np.sin(sun_altitude_rad) * np.cos(slope_radians) +
            np.cos(sun_altitude_rad) * np.sin(slope_radians) *
            np.cos(sun_azimuth_rad - aspect_radians)
        )
        
        # Normalize to 0-255 range
        hillshade = np.clip(hillshade * 255, 0, 255).astype(np.uint8)
        
        results['arrays']['hillshade'] = hillshade
        
        results['statistics']['hillshade'] = {
            'sun_azimuth': hillshade_azimuth,
            'sun_altitude': hillshade_altitude,
            'mean_value': float(np.mean(hillshade[~np.isnan(hillshade)])),
            'value_range': (int(np.min(hillshade[~np.isnan(hillshade)])), 
                           int(np.max(hillshade[~np.isnan(hillshade)])))
        }
        
        results['metrics_calculated'].append('hillshade')
        print(f"     ✅ Hillshade (sun: {hillshade_azimuth}°az, {hillshade_altitude}°alt)")
    
    if 'curvature' in metrics:
        print(f"   📈 Calculating curvature...")
        
        # Calculate second derivatives
        dxx = np.gradient(dx, cell_size, axis=1)
        dyy = np.gradient(dy, cell_size, axis=0)
        dxy = np.gradient(dx, cell_size, axis=0)
        
        # Profile curvature (curvature in the direction of steepest slope)
        numerator = dxx * dx**2 + 2 * dxy * dx * dy + dyy * dy**2
        denominator = (dx**2 + dy**2) * np.sqrt(1 + dx**2 + dy**2)**3
        profile_curvature = np.where(denominator != 0, numerator / denominator, 0)
        
        # Plan curvature (curvature perpendicular to the direction of steepest slope)
        numerator = dxx * dy**2 - 2 * dxy * dx * dy + dyy * dx**2
        denominator = (dx**2 + dy**2) * np.sqrt(1 + dx**2 + dy**2)
        plan_curvature = np.where(denominator != 0, numerator / denominator, 0)
        
        results['arrays']['profile_curvature'] = profile_curvature
        results['arrays']['plan_curvature'] = plan_curvature
        
        # Calculate statistics
        valid_profile = profile_curvature[~np.isnan(profile_curvature)]
        valid_plan = plan_curvature[~np.isnan(plan_curvature)]
        
        if len(valid_profile) > 0 and len(valid_plan) > 0:
            results['statistics']['curvature'] = {
                'profile_mean': float(np.mean(valid_profile)),
                'profile_std': float(np.std(valid_profile)),
                'plan_mean': float(np.mean(valid_plan)),
                'plan_std': float(np.std(valid_plan))
            }
    
    print(f"\n🎉 Topographic metrics calculation complete!")
    print(f"   📊 Computed {len(results['metrics'])} metric types")
    print(f"   📏 DEM resolution: {cell_size:.2f}m" if cell_size else "   📏 DEM resolution: auto-detected")
    
    return results

## 💡 Understanding Topographic Metrics

### Key Concepts:

**Slope:**
- **Definition**: Rate of elevation change, expressed in degrees or percent
- **Applications**: Erosion risk, accessibility analysis, habitat modeling
- **Calculation**: Uses elevation differences between neighboring cells

**Aspect:**
- **Definition**: Direction of steepest descent, expressed in degrees from north
- **Applications**: Solar radiation, snow accumulation, vegetation patterns
- **Range**: 0-360 degrees (0 = north, 90 = east, 180 = south, 270 = west)

**Hillshade:**
- **Definition**: Illumination simulation from a light source
- **Applications**: Terrain visualization, cartographic enhancement
- **Parameters**: Sun azimuth (direction) and altitude (elevation angle)

**Curvature:**
- **Profile Curvature**: Curvature in direction of steepest descent
- **Plan Curvature**: Curvature perpendicular to direction of steepest descent
- **Applications**: Water flow modeling, landform classification

## 🎯 Your Task: Implement and Test

**Requirements:**
1. **Implement the function** exactly as shown above
2. **Handle different DEM data types** (integer and floating-point elevation)
3. **Manage nodata values** properly in calculations
4. **Support multiple output formats** (arrays, files, or both)
5. **Validate elevation data** and handle edge cases
6. **Provide comprehensive statistics** for all computed metrics

**Key Implementation Points:**
- Use appropriate algorithms for terrain derivative calculations
- Handle DEM edge effects and boundary conditions
- Implement efficient processing for large datasets
- Provide meaningful progress feedback
- Return structured results with metadata

**Testing Strategy:**
```python
# Test different scenarios:
# 1. High-resolution local DEM
# 2. Low-resolution regional DEM
# 3. DEM with nodata values
# 4. Flat terrain areas
# 5. Mountainous terrain
# 6. Different output options
```

## 🔧 Testing Your Implementation

Run the official tests to verify your function works correctly:

```bash
cd /workspaces/your-repo
python -m pytest tests/test_advanced_rasterio_analysis.py::test_calculate_topographic_metrics -v
```

### Additional Testing Ideas:
```python
# Test with real elevation data
results = calculate_topographic_metrics(
    dem_path='srtm_elevation.tif',
    metrics=['slope', 'aspect', 'hillshade', 'curvature'],
    output_dir='terrain_analysis/'
)

# Validate slope calculations
assert 0 <= results['metrics']['slope'].max() <= 90
assert 0 <= results['metrics']['aspect'].max() <= 360
```

## 📚 Professional Applications

### Real-World Use Cases:

**Environmental Analysis:**
- Watershed delineation and hydrological modeling
- Erosion risk assessment and soil conservation planning
- Habitat suitability modeling for wildlife species

**Infrastructure Planning:**
- Road and trail route optimization
- Solar panel installation site assessment
- Telecommunications tower placement analysis

**Natural Hazards:**
- Landslide susceptibility mapping
- Flood risk modeling and inundation analysis
- Avalanche risk assessment in mountainous areas

**Agriculture:**
- Precision agriculture and variable rate applications
- Drainage design and water management
- Microclimate analysis for crop planning

### Industry Standards:
- **DEM Resolution**: 1m (LiDAR), 10m (local), 30m (regional), 90m (global)
- **Slope Classes**: Flat (<2°), Gentle (2-5°), Moderate (5-15°), Steep (15-35°), Very Steep (>35°)
- **Accuracy Requirements**: Vary by application (survey-grade vs. planning-level)
- **Data Standards**: USGS, SRTM, ASTER GDEM, local LiDAR products

## 🚀 Next Steps

Once this function works and passes the tests, move on to:
- **Function 2**: `calculate_vegetation_indices()` - Learn to analyze multispectral imagery for vegetation health assessment

**This completes the foundational terrain analysis skills!**

## 🎓 Real-World Applications

The topographic analysis techniques you've mastered are used for:
- **Hydrological modeling**: Understanding water flow and drainage patterns
- **Geomorphological analysis**: Classifying landforms and understanding landscape evolution
- **Environmental assessment**: Evaluating terrain influence on ecological processes
- **Engineering applications**: Site suitability and infrastructure design
- **Climate modeling**: Understanding topographic effects on weather patterns

**Excellent work on mastering terrain analysis fundamentals! 🍀**